In [9]:
from dataInput import *
from CV import *
from TwoStageSP import *
from FA import *
import numpy as np
import pandas as pd

In [10]:
inputParams = inputParams(1,1,57,1000);
solveParams = solveParams(100000,200,100000,10800,1e-4);

In [11]:
intensityFile = 'data/synthetic/intensity.csv';
locationFile = 'data/synthetic/location.csv';
landfallFile = 'data/synthetic/landfall_7.csv';

In [12]:
hurricaneDataSet = hurricaneInput(intensityFile, locationFile, landfallFile, inputParams)

In [13]:
netNodesFile = 'data/synthetic/nodes.csv';
netParamsFile = 'data/synthetic/netParams.csv';
networkDataSet = networkInput(3,20,0.05,netNodesFile,netParamsFile,hurricaneDataSet)

In [14]:
osfname = "./data/synthetic/OOS" + str(inputParams.k_init) + ".csv"
OS_paths = pd.read_csv(osfname).values  # Read the out-of-sample file

In [15]:
static_2SSP_eval(networkDataSet, hurricaneDataSet, inputParams, solveParams, osfname)

training LB = 10028.169702887639
training UB = 10022.245233337837
static 2SSP....
μ ± 1.96*σ/√NS = 9210.644836893458 ± 1274.534878589923


[[6742.605084474061,
  6912.970143022567,
  6649.818820929583,
  6123.516176743617,
  7195.941452031507,
  6742.605084474061,
  6798.861314262002,
  6926.102550408756,
  7195.941452031507,
  7109.493593544469,
  5860.364854650633,
  7195.941452031507,
  6926.102550408756,
  7195.941452031507,
  6649.818820929583,
  7231.661053241846,
  6926.102550408756,
  6533.498023730762,
  6123.516176743617,
  5860.364854650633,
  6656.263648786005,
  6123.516176743617,
  5860.364854650633,
  6123.516176743617,
  6386.667498836599,
  6798.861314262002,
  6123.516176743617,
  7299.701227053043,
  6123.516176743617,
  6386.667498836599,
  7231.661053241846,
  6477.5877356079145,
  6778.757872120026,
  6123.516176743617,
  7176.651560445164,
  6926.102550408756,
  5860.364854650633,
  7765.752364355526,
  5860.364854650633,
  6649.818820929583,
  6123.516176743617,
  5860.364854650633,
  6926.102550408756,
  7231.661053241846,
  8436.548821099877,
  7176.121465115551,
  6926.102550408756,
  6778.75787

In [16]:
RH_2SSP_eval(networkDataSet, hurricaneDataSet, inputParams, solveParams, osfname)

RH 2SSP....
μ ± 1.96*σ/√NS = 7253.629394551722 ± 140.0155997668221


[array([[ 6649.81882093,   263.15132209,   263.15132209, ...,
         -1052.60528837,     0.        ,     0.        ],
        [ 6649.81882093,   263.15132209,   263.15132209, ...,
           263.15132209, -1052.60528837,     0.        ],
        [ 6649.81882093,   263.15132209,   263.15132209, ...,
         -1052.60528837,     0.        ,     0.        ],
        ...,
        [ 6649.81882093,  7530.0615276 ,   526.74206307, ...,
         -2106.96825227,     0.        ,     0.        ],
        [ 6649.81882093,   263.15132209,   263.15132209, ...,
             0.        ,     0.        ,     0.        ],
        [ 6649.81882093,   263.15132209,   263.15132209, ...,
         -1052.60528837,     0.        ,     0.        ]]),
 7253.629394551722,
 7113.6137947849,
 7393.6449943185435,
 349.2844889163971]